In [10]:
import os
import pandas as pd 
import numpy as np
import geopandas as gpd

from pyproj import Transformer
from pyproj import CRS, Proj

from osgeo import gdal 
import rasterio
from rasterio.windows import from_bounds
from shapely.geometry import Point

In [ ]:
# Load the generation data
path = os.getcwd() + "\\Raw_Spatial_Data\\Gen 2016-2023_vs edit.xlsx"
gen_data_wet = pd.read_excel(path,"2022")
gen_data_dry = pd.read_excel(path,"2019")

### Plant Data

In [6]:
# Load the plant data (head, location, etc.)
path = os.getcwd() + "\\Raw_Spatial_Data\\19.7.2024-NEW UPDATED_Data_lao_231223_NPP_coordinate.xlsx"
data = pd.read_excel(path,'NPDP power plant info')

In [ ]:
# PREP OF PLANT DATA
data = data[['SNo', 'East E', 'North N', 'Status', 'PP name', 'New Ose_Name','Fuel Type',
       'Province', 'Region', 'Total capacity (MW)',
       'Domestic Capacity (MW)', 'Export Capacity (MW)',
       'Expected Generation (GWh)',
       'total theoretical possible generation (local) GWh', 'COD (Year)',
       'Exporting country country', 'Head Hydraulic (m)']]

data['East E'] = pd.to_numeric(data['East E'], errors='coerce')
data['North N'] = pd.to_numeric(data['North N'], errors='coerce')

data = data.dropna(subset=['East E', 'North N'])

transformer = Transformer.from_crs("epsg:32648", "epsg:4326", always_xy=True)

def convert_coordinates(easting, northing):
    longitude, latitude = transformer.transform(easting, northing)
    return latitude, longitude

converted_coords = data.apply(
    lambda row: convert_coordinates(row['East E'], row['North N']),
    axis=1
)

data = data.copy()
data[['Latitude', 'Longitude']] = pd.DataFrame(converted_coords.tolist(), index=data.index)
data[['East E','North N','Latitude','Longitude']]

data = data[['SNo', 'New Ose_Name', 'Status','Fuel Type','Latitude','Longitude','Total capacity (MW)','Domestic Capacity (MW)',
             'Expected Generation (GWh)','total theoretical possible generation (local) GWh',
             'COD (Year)','Head Hydraulic (m)']]
data = data.rename(columns={"Total capacity (MW)": "capacity",
                                   "COD (Year)": "COD",
                                   "Head Hydraulic (m)": "head"})

data = data[data['Fuel Type'].isin(['Run - Off', 'Reservoir '])]
data['capacity'] = pd.to_numeric(data['capacity'], errors='raise')

data['Fuel Type'] = data['Fuel Type'].str.replace("Reservoir ", "Reservoir")
data.to_excel("hydropower_list_seasonal.xlsx", index=False)

### Hydrological Network

In [21]:
# Coordinate Reference Systems
wgs84 = CRS("EPSG:4326")
merc = CRS("EPSG:3395")
merc_pro = Proj("EPSG:3395")

class LocalHydroNetwork:
    def __init__(self, dem_path, flow_path, flow_acc_path):
        self.dem = rasterio.open(dem_path)
        self.flow = rasterio.open(flow_path)
        self.flow_acc = rasterio.open(flow_acc_path)

    def create_local_network(self, point, buffer=0.005):
        minx, miny = point.x - buffer, point.y - buffer
        maxx, maxy = point.x + buffer, point.y + buffer

        window = rasterio.windows.from_bounds(minx, miny, maxx, maxy, self.dem.transform)
        dem_local = self.dem.read(1, window=window)
        flow_local = self.flow.read(1, window=window)
        flow_acc_local = self.flow_acc.read(1, window=window)

        transform = rasterio.windows.transform(window, self.dem.transform)

        nodes, arcs = self._extract_nodes_arcs(dem_local, flow_local, flow_acc_local, transform)
        return nodes, arcs

    def _extract_nodes_arcs(self, dem_local, flow_local, flow_acc_local, transform):
        nodes = []
        arcs = []
        height, width = dem_local.shape
        node_index_map = {}

        for i in range(height):
            for j in range(width):
                x, y = transform * (j, i)
                elevation = dem_local[i, j]
                flow_dir = flow_local[i, j]
                flow_acc = flow_acc_local[i, j]

                current_node = (i, j)
                if current_node not in node_index_map:
                    node_id = len(nodes)
                    node = {
                        'id': node_id,
                        'x': x,
                        'y': y,
                        'elevation': elevation,
                        'flow_acc': flow_acc,
                        'arcs': []
                    }
                    nodes.append(node)
                    node_index_map[current_node] = node_id
                else:
                    node_id = node_index_map[current_node]

                if flow_dir > 0:
                    next_i, next_j = self._flow_direction_to_indices(i, j, flow_dir)
                    if 0 <= next_i < height and 0 <= next_j < width:
                        next_node = (next_i, next_j)
                        if next_node not in node_index_map:
                            next_node_id = len(nodes)
                            next_x, next_y = transform * (next_j, next_i)
                            next_elevation = dem_local[next_i, next_j]
                            next_flow_acc = flow_acc_local[next_i, next_j]

                            next_node_data = {
                                'id': next_node_id,
                                'x': next_x,
                                'y': next_y,
                                'elevation': next_elevation,
                                'flow_acc': next_flow_acc,
                                'arcs': []
                            }
                            nodes.append(next_node_data)
                            node_index_map[next_node] = next_node_id
                        else:
                            next_node_id = node_index_map[next_node]

                        arc = {
                            'start_node': node_id,
                            'end_node': next_node_id,
                            'length': np.hypot(next_i - i, next_j - j)
                        }
                        nodes[node_id]['arcs'].append(arc)
                        arcs.append(arc)

        return nodes, arcs

    def _flow_direction_to_indices(self, i, j, direction):
        if direction == 1:  # East
            return i, j + 1
        elif direction == 2:  # Southeast
            return i + 1, j + 1
        elif direction == 4:  # South
            return i + 1, j
        elif direction == 8:  # Southwest
            return i + 1, j - 1
        elif direction == 16:  # West
            return i, j - 1
        elif direction == 32:  # Northwest
            return i - 1, j - 1
        elif direction == 64:  # North
            return i - 1, j
        elif direction == 128:  # Northeast
            return i - 1, j + 1
        else:
            return i, j  # No direction

    def calculate_head(self, nodes, arcs):
        max_head = 0
        for arc in arcs:
            start_node = nodes[arc['start_node']]
            end_node = nodes[arc['end_node']]
            head = start_node['elevation'] - end_node['elevation']
            if head > max_head:
                max_head = head
        return max_head

    def process_hydropower_plants(self, df, buffer):
        df['head_calculated'] = pd.NA
        for index, row in df.iterrows():
            point = Point(row['Longitude'], row['Latitude'])
            nodes, arcs = self.create_local_network(point, buffer)
            head = self.calculate_head(nodes, arcs)
            df.at[index, 'head_calculated'] = head
        df['head_difference'] = df['head'] - df['head_calculated']
        return df

    def optimize_buffer(self, df, buffer_range):
        best_mae = float('inf')
        best_std = float('inf')
        best_buffer = None
        best_df = None
        
        for buffer in buffer_range:
            df_processed = self.process_hydropower_plants(df.copy(), buffer)
            mae = df_processed['head_difference'].abs().mean()
            std = df_processed['head_difference'].std()
            if mae + std < best_mae + best_std:
                best_mae = mae
                best_std = std
                best_buffer = buffer
                best_df = df_processed
        
        print(f"Optimal Buffer: {best_buffer}")
        print(f"Mean Absolute Error: {best_mae}")
        print(f"Standard Deviation: {best_std}")
        
        return best_df, best_buffer, best_mae, best_std

    def optimize_for_fuel_types(self, df, buffer_range):
        # Optimize only for rows with existing head values
        reservoir_df = df[df['Fuel Type'] == 'Reservoir'].dropna(subset=['head']).copy()
        runoff_df = df[df['Fuel Type'] == 'Run - Off'].dropna(subset=['head']).copy()

        print("\nOptimizing for Reservoir plants...")
        optimized_reservoir_df, optimal_reservoir_buffer, reservoir_mae, reservoir_std = self.optimize_buffer(reservoir_df, buffer_range)
        
        print("\nOptimizing for Run - Off plants...")
        optimized_runoff_df, optimal_runoff_buffer, runoff_mae, runoff_std = self.optimize_buffer(runoff_df, buffer_range)

        return {
            'reservoir': {
                'df': optimized_reservoir_df, 
                'buffer': optimal_reservoir_buffer, 
                'mae': reservoir_mae, 
                'std': reservoir_std
            },
            'runoff': {
                'df': optimized_runoff_df, 
                'buffer': optimal_runoff_buffer, 
                'mae': runoff_mae, 
                'std': runoff_std
            }
        }

    def calculate_and_fill_missing_heads(self, df, reservoir_buffer, runoff_buffer):
        # Process all rows with the optimized buffer sizes
        df_reservoir = self.process_hydropower_plants(df[df['Fuel Type'] == 'Reservoir'].copy(), reservoir_buffer)
        df_runoff = self.process_hydropower_plants(df[df['Fuel Type'] == 'Run - Off'].copy(), runoff_buffer)

        # Combine results back into the original DataFrame
        df_combined = pd.concat([df_reservoir, df_runoff])

        # Fill missing head values in the original DataFrame
        for index, row in df_combined.iterrows():
            if pd.isna(df.at[index, 'head']):
                df.at[index, 'head'] = df_combined.at[index, 'head_calculated']

        return df

In [20]:
path = os.getcwd()
dem_path = path + "\\Raw_Spatial_Data\\hydro_data\\dem_con_asia.tif"
flow_path = path + "\\Raw_Spatial_Data\\hydro_data\\flow_asia.tif"
flow_acc_path = path + "\Raw_Spatial_Data\\hydro_data\\as_acc_3s.tif"
xlsx_file = "hydropower_list_seasonal.xlsx"

buffer_range = np.linspace(0.00025, 0.005, 101)
hydro_network = LocalHydroNetwork(dem_path, flow_path, flow_acc_path)

df = pd.read_excel(xlsx_file)
results = hydro_network.optimize_for_fuel_types(df, buffer_range)

plant_data = hydro_network.calculate_and_fill_missing_heads(df, results['reservoir']['buffer'], results['runoff']['buffer'])


Optimizing for Reservoir plants...
Optimal Buffer: 0.003385
Mean Absolute Error: 26.0828125
Standard Deviation: 29.08527294557416

Optimizing for Run - Off plants...
Optimal Buffer: 0.003195
Mean Absolute Error: 45.85
Standard Deviation: 84.40090732862258


### Merge Data

In [ ]:
merged_data_wet = pd.merge(gen_data_wet, plant_data, left_on='OSeMOSYS tech', right_on='New Ose_Name', how='inner')
merged_data_wet = pd.merge(gen_data_dry, plant_data, left_on='OSeMOSYS tech', right_on='New Ose_Name', how='inner')